Geographical Information Science 2021
Analysis of data on covid19
Simon Danny Pettinella

In [ ]:
#import libraries
import geopandas
import pandas as pd
import io
import requests
import folium
import json
import branca.colormap as cm
import warnings
warnings.filterwarnings('ignore')

Map of Italy about covid19
In order to have always updated and official data, it was decided to use the ministerial dataset containing all the necessary information relating to covid19 for each Italian region.
“https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-latest.csv”

In [ ]:
#upload of the geojson file containing the borders of the Italian regions
df = geopandas.read_file("italia.geojson")
df.rename( columns = {"NOME_REG": "Regione"}, inplace = True)

In [ ]:
#loading of the ministerial dataset (update at 5 pm)
url_covid19="https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-latest.csv"
url_covid=requests.get(url_covid19).content
dati_covid19=pd.read_csv(io.StringIO(url_covid.decode()))

In [ ]:
dati_covid19

In [ ]:
#clean dataset
dati_covid19.drop(['stato','data','codice_regione','note','ingressi_terapia_intensiva','note','note_test','note_casi','codice_nuts_2',
                  'codice_nuts_1','tamponi_test_antigenico_rapido','casi_da_sospetto_diagnostico','casi_da_screening','tamponi_test_molecolare',
                   'totale_positivi_test_antigenico_rapido','variazione_totale_positivi','ricoverati_con_sintomi','terapia_intensiva','nuovi_positivi','totale_positivi_test_molecolare','casi_testati'],axis=1,inplace=True)
dati_covid19.rename( columns = {"denominazione_regione": "Regione"}, inplace = True)

In [ ]:
#management of the autonomous provinces of Trentino Alto Adige
dati_covid19.set_index('Regione',inplace=True)
trentino = pd.Series(data=dati_covid19.iloc[11]+dati_covid19.iloc[12], name ='Trentino-Alto Adige')
dati_covid19 = dati_covid19.append(trentino)
dati_covid19.drop(dati_covid19.index[[11,12]], inplace=True)
dati_covid19.reset_index(inplace=True)
dati_covid19.iloc[[19],[1]] = '46.499335'
dati_covid19.iloc[[19],[2]] = '11.356624'

In [ ]:
dati_covid19

In [ ]:
#Creation of the geopandas dataframe necessary to create the map
geo_df=df.merge(dati_covid19,on="Regione")

In [ ]:
geo_df

In [ ]:
map = folium.Map(location = [43,12], zoom_start = 6, tiles=None) #creazione della mappa e localizzazione
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(map) #aggiunta dei livelli

# Creating choropleth map
map.choropleth(geo_data=geo_df, #inserimento df geospaziale
             #name='covid19',         
             data=geo_df, #inserimento df dati
             columns=['Regione','totale_positivi'], #selezione delle colonne 
             key_on='feature.properties.Regione', #selezione dei dati json necessari
             fill_color='YlOrRd', #colore mappa
             fill_opacity=0.6, #trasparenza zone
             line_opacity=0.8, #trasparenza linea delimitante
             legend_name='Numero totale di positivi', #legenda
             smooth_factor=0,     
             highlight=True)

style_function = lambda x: {'color':'black','fillColor':'transparent','weight':0.5}
highlight_function = lambda x: {'weight':3,'fillColor':'grey'}

SuburbName = folium.features.GeoJson(
    geo_df,
    style_function=style_function,
    highlight_function=highlight_function,
    control=False,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Regione'
                ,'totale_positivi'
                ,'dimessi_guariti'
                ,'deceduti'
               ],
        aliases=['Regione: '
                ,'Positivi: '
                 ,'Guariti:'
                 ,'Deceduti:'
                 ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

map.add_child(SuburbName)
map.keep_in_front(SuburbName)

map

#Mappe dell’Italia relative ai vaccini
#Al fine di disporre di dati sempre aggiornati ed ufficiali, 
#si è scelto di utilizzare il dataset ministeriale contenente tutte 
#le informazioni necessarie relative ai vaccini per il covid19 per ogni regione italiana. 
#“https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv”

In [ ]:
#loading of the other ministerial dataset (update at 5 pm)
url_vaccini="https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"
s_vaccini=requests.get(url_vaccini).content
vaccini=pd.read_csv(io.StringIO(s_vaccini.decode('utf-8')))

In [ ]:
vaccini

In [ ]:
#clean dataset
vaccini.drop(['data','area','N1','N2','ISTAT'],axis=1,inplace=True)
vaccini = vaccini.groupby(['reg','forn','eta']).agg({'d1': ['sum'],'d2':['sum']})
vaccini.reset_index(inplace=True)
vaccini.rename( columns = {"nome_area": "Regione"}, inplace = True)
vaccini.iloc[431:503, 0] = "Trentino-Alto Adige"
vaccini.iloc[682:716, 0] = "Valle d'Aosta"
vaccini.iloc[179:215, 0] = "Friuli Venezia Giulia"

In [ ]:
vaccini

In [ ]:
grou = vaccini.groupby(['reg','eta','forn']).sum().reset_index()
grou.columns = grou.columns.droplevel(1)
grou['totale_dosi'] = grou['d1'] + grou['d2']
grou.rename( columns = {"reg": "Regione"}, inplace = True)
grou.rename( columns = {"d1": "prima_dose"}, inplace = True)
grou.rename( columns = {"d2": "seconda_dose"}, inplace = True)
grou.rename( columns = {"forn": "fornitore"}, inplace = True)
grou

In [ ]:
#definition of the datasets based on the age range
fascia_eta_05_11 = grou[grou['eta'] == '05-11']
fascia_eta_12_19 = grou[grou['eta'] == '12-19']
fascia_eta_20_29 = grou[grou['eta'] == '20-29']
fascia_eta_30_39 = grou[grou['eta'] == '30-39']
fascia_eta_40_49 = grou[grou['eta'] == '40-49']
fascia_eta_50_59 = grou[grou['eta'] == '50-59']
fascia_eta_60_69 = grou[grou['eta'] == '60-69']
fascia_eta_70_79 = grou[grou['eta'] == '70-79']
fascia_eta_80_89 = grou[grou['eta'] == '80-89']
fascia_eta_90 = grou[grou['eta'] == '90+']

In [ ]:
#definition of the total doses administered for each age group
grou_05_11 = fascia_eta_05_11.groupby(['Regione']).sum().reset_index()
grou_05_11['totale_dosi'] = grou_05_11['prima_dose'] + grou_05_11['seconda_dose']
grou_12_19 = fascia_eta_12_19.groupby(['Regione']).sum().reset_index()
grou_12_19['totale_dosi'] = grou_12_19['prima_dose'] + grou_12_19['seconda_dose']
grou_20_29 = fascia_eta_20_29.groupby(['Regione']).sum().reset_index()
grou_20_29['totale_dosi'] = grou_20_29['prima_dose'] + grou_20_29['seconda_dose']
grou_30_39 = fascia_eta_30_39.groupby(['Regione']).sum().reset_index()
grou_30_39['totale_dosi'] = grou_30_39['prima_dose'] + grou_30_39['seconda_dose']
grou_40_49 = fascia_eta_40_49.groupby(['Regione']).sum().reset_index()
grou_40_49['totale_dosi'] = grou_40_49['prima_dose'] + grou_40_49['seconda_dose']
grou_50_59 = fascia_eta_50_59.groupby(['Regione']).sum().reset_index()
grou_50_59['totale_dosi'] = grou_50_59['prima_dose'] + grou_50_59['seconda_dose']
grou_60_69 = fascia_eta_60_69.groupby(['Regione']).sum().reset_index()
grou_60_69['totale_dosi'] = grou_60_69['prima_dose'] + grou_60_69['seconda_dose']
grou_70_79 = fascia_eta_70_79.groupby(['Regione']).sum().reset_index()
grou_70_79['totale_dosi'] = grou_70_79['prima_dose'] + grou_70_79['seconda_dose']
grou_80_89 = fascia_eta_80_89.groupby(['Regione']).sum().reset_index()
grou_80_89['totale_dosi'] = grou_80_89['prima_dose'] + grou_80_89['seconda_dose']
grou_90 = fascia_eta_90.groupby(['Regione']).sum().reset_index()
grou_90['totale_dosi'] = grou_90['prima_dose'] + grou_90['seconda_dose']

In [ ]:
#Creation of the geopandas dataframe needed to create the map
vaccini_df_12_19 =df.merge(grou_12_19,on="Regione")
vaccini_df_20_29 =df.merge(grou_20_29,on="Regione")
vaccini_df_30_39 =df.merge(grou_30_39,on="Regione")
vaccini_df_40_49 =df.merge(grou_40_49,on="Regione")
vaccini_df_50_59 =df.merge(grou_50_59,on="Regione")
vaccini_df_60_69 =df.merge(grou_60_69,on="Regione")
vaccini_df_70_79 =df.merge(grou_70_79,on="Regione")
vaccini_df_80_89 =df.merge(grou_80_89,on="Regione")
vaccini_df_90 =df.merge(grou_90,on="Regione")

Map of Italy relating to vaccines divided by age group 
For each Italian region, it is possible to view - through the drop-down menu on the right - the data relating to the quantity of vaccines (first dose, second dose, total doses) administered for each group. 'age (12-19.20-29.30-39.40-49.50-59.60-69.70-79.80-89.90 +).

In [ ]:
m = folium.Map([43, 12], zoom_start=5,tiles=None,overlay=False)

ch1 = folium.Choropleth(
    geo_data=vaccini_df_12_19,
    name='12-19',
    data=vaccini_df_12_19,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    line_opacity=0.2,
    overlay=False,
    legend_name = 'Totale dosi di vaccino per fascia di età 12-19',
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
    
g = folium.features.GeoJson(vaccini_df_12_19,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch1)

ch2 = folium.Choropleth(
    geo_data=vaccini_df_20_29,
    name='20-29',
    data=vaccini_df_20_29,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi di vaccino per fascia di età 20-29',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)

g1 = folium.features.GeoJson(vaccini_df_20_29,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch2)

ch3 = folium.Choropleth(
    geo_data=vaccini_df_30_39,
    name='30-39',
    data=vaccini_df_30_39,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi di vaccino per fascia di età 30-39',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)

g2 = folium.features.GeoJson(vaccini_df_30_39,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch3)

ch4 = folium.Choropleth(
    geo_data=vaccini_df_40_49,
    name='40-49',
    data=vaccini_df_40_49,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi di vaccino per fascia di età 40-49',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)

g3 = folium.features.GeoJson(vaccini_df_40_49,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch4)


ch5 = folium.Choropleth(
    geo_data=vaccini_df_50_59,
    name='50-59',
    data=vaccini_df_50_59,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi di vaccino per fascia di età 50-59',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
g4 = folium.features.GeoJson(vaccini_df_50_59,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch5)

ch6 = folium.Choropleth(
    geo_data=vaccini_df_60_69,
    name='60-69',
    data=vaccini_df_60_69,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi di vaccino per fascia di età 60-69',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
g5 = folium.features.GeoJson(vaccini_df_60_69,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch6)

ch7 = folium.Choropleth(
    geo_data=vaccini_df_70_79,
    name='70-79',
    data=vaccini_df_70_79,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi di vaccino per fascia di età 70-79',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
g6 = folium.features.GeoJson(vaccini_df_70_79,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch7)

ch8 = folium.Choropleth(
    geo_data=vaccini_df_80_89,
    name='80-89',
    data=vaccini_df_80_89,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi di vaccino per fascia di età 80-89',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
g7 = folium.features.GeoJson(vaccini_df_80_89,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch8)

ch9 = folium.Choropleth(
    geo_data=vaccini_df_90,
    name='90+',
    data=vaccini_df_90,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi di vaccino per fascia di età 90+',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
g8 = folium.features.GeoJson(vaccini_df_90,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch9)
folium.LayerControl(collapsed=False).add_to(m)
m

In [ ]:
#clean dataset
tipo_vacc = grou.groupby(['Regione','fornitore']).sum().reset_index()
totale_vacc = tipo_vacc.groupby(['Regione']).sum().reset_index()

In [ ]:
#Creation of the geopandas dataframe needed to create the map
vaccini_df_totali =df.merge(totale_vacc,on="Regione")

In [ ]:
tipo_vacc_mod = tipo_vacc[tipo_vacc['fornitore'] == 'Moderna']
tipo_vacc_astr = tipo_vacc[tipo_vacc['fornitore'] == 'Vaxzevria (AstraZeneca)']
tipo_vacc_pfz = tipo_vacc[tipo_vacc['fornitore'] == 'Pfizer/BioNTech']
tipo_vacc_jan = tipo_vacc[tipo_vacc['fornitore'] == 'Janssen']

In [ ]:
##Creation of geopandas dataframes
vaccini_df_moderna =df.merge(tipo_vacc_mod,on="Regione")
vaccini_df_astrazeneca =df.merge(tipo_vacc_astr,on="Regione")
vaccini_df_pfizer =df.merge(tipo_vacc_pfz,on="Regione")
vaccini_df_janssen =df.merge(tipo_vacc_jan,on="Regione")

In [ ]:
#loading of the csv file containing the number of inhabitants of each Italian region
num_abit = pd.read_csv("numero_abitanti_italia.csv",sep=';')

In [ ]:
#creation of the dataset containing the number of the NON-vaccinated population for each Italian region
no_vax = vaccini_df_totali.merge(num_abit, on='Regione')
no_vax.drop(['totale_dosi','seconda_dose'],axis=1,inplace=True)
no_vax['non vaccinati'] = (no_vax['Popolazione'] - no_vax['prima_dose'])

Map of Italy relating to vaccines divided by type of vaccine
For each Italian region, it is possible to view - via the drop-down menu on the right - the data relating to the quantity of vaccines (first dose, second dose, total doses) administered for each brand vaccine (Moderna, Pfizer, Astrazeneca and Jannsen).

In [ ]:
m = folium.Map([43, 12], zoom_start=5,tiles=None,overlay=False)

ch1 = folium.Choropleth(
    geo_data=vaccini_df_moderna,
    name='Moderna',
    data=vaccini_df_moderna,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    line_opacity=0.2,
    overlay=False,
    legend_name = 'Totale dosi per vaccino Moderna',
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
    
g = folium.features.GeoJson(vaccini_df_moderna,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch1)

ch2 = folium.Choropleth(
    geo_data=vaccini_df_pfizer,
    name='Pfizer',
    data=vaccini_df_pfizer,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='BuPu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi per vaccino Pfizer',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)

g1 = folium.features.GeoJson(vaccini_df_pfizer,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch2)

ch3 = folium.Choropleth(
    geo_data=vaccini_df_janssen,
    name='Janssen',
    data=vaccini_df_janssen,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='PuOr',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi per vaccino Janssen',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)

g2 = folium.features.GeoJson(vaccini_df_janssen,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch3)

ch4 = folium.Choropleth(
    geo_data=vaccini_df_astrazeneca,
    name='Astrazeneca',
    data=vaccini_df_astrazeneca,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='RdBu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi per vaccino Astrazeneca',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)

g3 = folium.features.GeoJson(vaccini_df_astrazeneca,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch4)


ch5 = folium.Choropleth(
    geo_data=vaccini_df_totali,
    name='Totali',
    data=vaccini_df_totali,
    columns=['Regione', 'totale_dosi'],
    key_on='feature.properties.Regione',
    fill_color='RdPu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi (senza distinzione di vaccino)',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
g4 = folium.features.GeoJson(vaccini_df_totali,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose'
                ,'seconda_dose'
                ,'totale_dosi'
               ],
                aliases=['Regione:'
                ,'Prima Dose:'
                 ,'Seconda Dose:'
                 ,'Totale Dosi:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch5)


ch6 = folium.Choropleth(
    geo_data=no_vax,
    name='Non vaccinati',
    data=no_vax,
    columns=['Regione', 'non vaccinati'],
    key_on='feature.properties.Regione',
    fill_color='RdPu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale non vaccinati',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
g5 = folium.features.GeoJson(no_vax,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'non vaccinati'
               ],
                aliases=['Regione:'
                 ,'Non vaccinati:'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch6)


folium.LayerControl(collapsed=False).add_to(m)
m

In [ ]:
vaccini_df_moderna_pop = vaccini_df_moderna.merge(num_abit, on='Regione')
vaccini_df_moderna_pop['prima_dose_pop'] = round((vaccini_df_moderna_pop['prima_dose']*100) / vaccini_df_moderna_pop['Popolazione'],2)
vaccini_df_moderna_pop['seconda_dose_pop'] = round((vaccini_df_moderna_pop['seconda_dose']*100) / vaccini_df_moderna_pop['Popolazione'],2)
vaccini_df_moderna_pop['totale_dosi_pop'] = round((vaccini_df_moderna_pop['totale_dosi']*100) / vaccini_df_moderna_pop['Popolazione'],2)

vaccini_df_pfizer_pop = vaccini_df_pfizer.merge(num_abit, on='Regione')
vaccini_df_pfizer_pop['prima_dose_pop'] = round((vaccini_df_pfizer_pop['prima_dose']*100) / vaccini_df_pfizer_pop['Popolazione'],2)
vaccini_df_pfizer_pop['seconda_dose_pop'] = round((vaccini_df_pfizer_pop['seconda_dose']*100) / vaccini_df_pfizer_pop['Popolazione'],2)
vaccini_df_pfizer_pop['totale_dosi_pop'] = round((vaccini_df_pfizer_pop['totale_dosi']*100) / vaccini_df_pfizer_pop['Popolazione'],2)

vaccini_df_janssen_pop = vaccini_df_janssen.merge(num_abit, on='Regione')
vaccini_df_janssen_pop['totale_dosi_pop'] = round((vaccini_df_janssen_pop['totale_dosi']*100) / vaccini_df_janssen_pop['Popolazione'],2)

vaccini_df_astrazeneca_pop = vaccini_df_astrazeneca.merge(num_abit, on='Regione')
vaccini_df_astrazeneca_pop['prima_dose_pop'] = round((vaccini_df_astrazeneca_pop['prima_dose']*100) / vaccini_df_astrazeneca_pop['Popolazione'],2)
vaccini_df_astrazeneca_pop['seconda_dose_pop'] = round((vaccini_df_astrazeneca_pop['seconda_dose']*100) / vaccini_df_astrazeneca_pop['Popolazione'],2)
vaccini_df_astrazeneca_pop['totale_dosi_pop'] = round((vaccini_df_astrazeneca_pop['totale_dosi']*100) / vaccini_df_astrazeneca_pop['Popolazione'],2)

vaccini_df_totali_pop = vaccini_df_totali.merge(num_abit, on='Regione')
vaccini_df_totali_pop['totale_dosi_pop'] = round((vaccini_df_totali['totale_dosi']*100)/num_abit['Popolazione'],2)


no_vax['no_vax_pop'] = round((no_vax['non vaccinati']*100)/no_vax['Popolazione'],2)

Map of Italy relating to vaccines divided by type of vaccine 
For each Italian region, it is possible to view - via the drop-down menu on the right - the data relating to the quantity of vaccines (first dose, second dose, total doses) administered for each brand vaccine (Moderna, Pfizer, Astrazeneca and Jannsen).

In [ ]:
m = folium.Map([43, 12], zoom_start=5,tiles=None,overlay=False)

ch1 = folium.Choropleth(
    geo_data=vaccini_df_moderna_pop,
    name='Moderna',
    data=vaccini_df_moderna_pop,
    columns=['Regione', 'totale_dosi_pop'],
    key_on='feature.properties.Regione',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    line_opacity=0.2,
    overlay=False,
    legend_name = '% Totale dosi per vaccino Moderna per popolazione',
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
    
g = folium.features.GeoJson(vaccini_df_moderna_pop,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose_pop'
                ,'seconda_dose_pop'
                ,'totale_dosi_pop'
               ],
                aliases=['Regione:'
                ,'Prima Dose: %'
                 ,'Seconda Dose: %'
                 ,'Totale Dosi: %'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch1)

ch2 = folium.Choropleth(
    geo_data=vaccini_df_pfizer_pop,
    name='Pfizer',
    data=vaccini_df_pfizer_pop,
    columns=['Regione', 'totale_dosi_pop'],
    key_on='feature.properties.Regione',
    fill_color='BuPu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='% Totale dosi per vaccino Pfizer per popolazione',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)

g1 = folium.features.GeoJson(vaccini_df_pfizer_pop,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose_pop'
                ,'seconda_dose_pop'
                ,'totale_dosi_pop'
               ],
                aliases=['Regione:'
                ,'Prima Dose: %'
                 ,'Seconda Dose: %'
                 ,'Totale Dosi: %'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch2)

ch3 = folium.Choropleth(
    geo_data=vaccini_df_janssen_pop,
    name='Janssen',
    data=vaccini_df_janssen_pop,
    columns=['Regione', 'totale_dosi_pop'],
    key_on='feature.properties.Regione',
    fill_color='PuOr',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='% Totale dosi per vaccino Janssen per popolazione',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)

g2 = folium.features.GeoJson(vaccini_df_janssen_pop,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'totale_dosi_pop'
               ],
                aliases=['Regione:'
                 ,'Totale Dosi: %'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch3)

ch4 = folium.Choropleth(
    geo_data=vaccini_df_astrazeneca_pop,
    name='Astrazeneca',
    data=vaccini_df_astrazeneca_pop,
    columns=['Regione', 'totale_dosi_pop'],
    key_on='feature.properties.Regione',
    fill_color='RdBu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='% Totale dosi per vaccino Astrazeneca per popolazione',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)

g3 = folium.features.GeoJson(vaccini_df_astrazeneca_pop,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'prima_dose_pop'
                ,'seconda_dose_pop'
                ,'totale_dosi_pop'
               ],
                aliases=['Regione:'
                ,'Prima Dose: %'
                 ,'Seconda Dose: %'
                 ,'Totale Dosi: %'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch4)


ch5 = folium.Choropleth(
    geo_data=vaccini_df_totali_pop,
    name='Totali',
    data=vaccini_df_totali_pop,
    columns=['Regione', 'totale_dosi_pop'],
    key_on='feature.properties.Regione',
    fill_color='RdPu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='% Totale dosi (senza distinzione di vaccino) per popolazione',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
g4 = folium.features.GeoJson(vaccini_df_totali_pop,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'totale_dosi_pop'
               ],
                aliases=['Regione: %'
                 ,'Totale Dosi: %'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch5)


ch6 = folium.Choropleth(
    geo_data=no_vax,
    name='Non vaccinati',
    data=no_vax,
    columns=['Regione', 'no_vax_pop'],
    key_on='feature.properties.Regione',
    fill_color='RdPu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='% Totale non vaccinati per popolazione',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(m)
g5 = folium.features.GeoJson(no_vax,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['Regione'
                ,'no_vax_pop'
               ],
                aliases=['Regione: %'
                 ,'Non vaccinati: %'
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch6)


folium.LayerControl(collapsed=False).add_to(m)
m

## Study of the covid19 situation in Germany


In [ ]:
#upload of the geojson file containing the borders of the countries of Germany
df_geo = geopandas.read_file("germania.geojson") #apertura del file geojson

In [ ]:
#loading the necessary data
dls = "https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Daten/Impfquotenmonitoring.xlsx?__blob=publicationFile"
resp = requests.get(dls)
output = open('Impfquotenmonitoring.xlsx', 'wb')
output.write(resp.content)
output.close()

In [ ]:
df = pd.read_excel('Impfquotenmonitoring.xlsx', sheet_name=None) #impostazione del file aperto come df
sheet_select = list(df.keys())[2] #selezione del foglio n.3
df = pd.read_excel('Impfquotenmonitoring.xlsx', sheet_name=sheet_select) #apertura df sul foglio interessato
df.to_csv('csvfile.csv', encoding='utf-8', index=False) #conversione da xlsx a csv

In [ ]:
df

In [ ]:
#clean dataset
df.rename( columns = {"Bundesland": "stati", 
          "mindestens einmal geimpft *": "Prima dose", 
          "Unnamed: 3": "Pfizer", 
          "Unnamed: 4": "Moderna", 
          "Unnamed: 5": "AstraZeneca", 
          "Unnamed: 6": "Janssen",
          "Unnamed: 7": "Novavax",},inplace = True)
df.drop(df.index[[0,19,20,21,22,23]], inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
df

In [ ]:
df['Pfizer'] = pd.to_numeric(df['Pfizer'],errors = 'coerce') #conversione valori in tipo numerico
df['Moderna'] = pd.to_numeric(df['Moderna'],errors = 'coerce')
df['AstraZeneca'] = pd.to_numeric(df['AstraZeneca'],errors = 'coerce')
df['Janssen'] = pd.to_numeric(df['Janssen'],errors = 'coerce')
df['Novavax'] = pd.to_numeric(df['Novavax'],errors = 'coerce')

df.rename(columns = {'stati' : 'name'}, inplace=True)

In [ ]:
#Creating the geopandas dataframe
final_df=df_geo.merge(df,on="name")

In [ ]:
map = folium.Map(location = [51.133481,10.018343], zoom_start = 5, tiles=None)

ch1 = folium.Choropleth(
    geo_data=final_df,
    name='Pfizer',
    data=final_df,
    columns=['name','Pfizer'], 
    key_on='feature.properties.name',
    fill_color='YlGn',
    nan_fill_color="black",
    fill_opacity=0.7,
    line_opacity=0.2,
    overlay=False,
    legend_name = 'Numero dosi vaccino Pfizer',
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(map)
    
g = folium.features.GeoJson(final_df,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(fields=['name','Pfizer'],
        aliases=['Stato: ','Totale Dosi: '],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch1)

ch2 = folium.Choropleth(
    geo_data=final_df,
    name='Moderna',
    data=final_df,
    columns=['name','Moderna'],
    key_on='feature.properties.name',
    fill_color='BuPu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi vaccino Moderna',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(map)

g1 = folium.features.GeoJson(final_df,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['name'
                ,'Moderna'
               ],
        aliases=['Stato: ','Totale Dosi: '],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch2)

ch3 = folium.Choropleth(
    geo_data=final_df,
    name='Janssen',
    data=final_df,
    columns=['name','Janssen'],
    key_on='feature.properties.name',
    fill_color='PuOr',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi vaccino Janssen',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(map)

g2 = folium.features.GeoJson(final_df,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['name'
                ,'Janssen'
                ],
        aliases=['Stato: '
                ,'Totale Dosi: '],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch3)

ch4 = folium.Choropleth(
    geo_data=final_df,
    name='AstraZeneca',
    data=final_df,
    columns=['name','AstraZeneca'],
    key_on='feature.properties.name',
    fill_color='RdBu',
    nan_fill_color="black",
    fill_opacity=0.7,
    legend_name='Totale dosi vaccino AstraZeneca',
    line_opacity=0.2,
    overlay=False,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    line_color='black').add_to(map)

g3 = folium.features.GeoJson(final_df,
               style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
               highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                control = False,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['name'
                ,'AstraZeneca'
               ],
        aliases=['Stato: '
                ,'Totale Dosi: '
                 ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"))).add_to(ch4)

folium.LayerControl(collapsed=False).add_to(map)
map